In [75]:
import pandas as pd
import plotly.express as px

# See: https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550
sheet_id = "1q8MB-H49pd1ojqZzMbs0ExbMkWKl0Ll0cyJ3_OpF3_g"
sheet_name = "Job_Search_Status"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url, usecols=range(0,11))

In [76]:
df


,Company,Industry,Position,Application Link,Location,Status,Source,App Date,My TODO,Vivian's TODO,Notes
0,Hudson River Trading,Finance,Middle office analyst - crypto,https://www.hudsonrivertrading.com/careers/job...,NYC,Interview,NYCDSA,4/19/2022,Wait for second round,She recommended me already,4/11 Interview. \nJaclyn/Jackie is the HR pers...
1,Triton.ai,Finance,Full-stack Software Engineer,Tony Carr (recruiter),NYC,Closed,Recruiter,4/18/2022,NaN,NaN,Wait for Tony. No response
2,JPMC,Finance,Data Scientist Associate,https://jpmc.fa.oraclecloud.com/hcmUI/Candidat...,Jersey City,Open,NYCDSA,4/20/2022,NaN,NaN,Sent Courtney resume to forward. Also applied ...
3,Haystacks.ai,Real Estate,SDE,NaN,NaN,Interview,NYCDSA,4/21/2022,Wait for tech assessment,She sent resume to them,They need SDEs. Call with Joe on Friday 4/29
4,Neiman Marcus Group,Retail,Data Scientist,https://www.linkedin.com/jobs/view/3021946314/...,Remote,Closed,NYCDSA,4/24/2022,NaN,NaN,They don't hire in WA
...,...,...,...,...,...,...,...,...,...,...,...
121,Twitter,Technology,Data Scientist,https://careers.twitter.com/en/work-for-twitte...,Seattle,Open,LinkedIn,5/2/2022,NaN,NaN,NaN
122,Substack,NaN,Data Analyst,https://boards.greenhouse.io/substack/jobs/400...,Remote,Open,LinkedIn,5/2/2022,NaN,NaN,hn
123,Guided.gg,Gaming,Data Scientist,https://jobs.lever.co/guilded/3a85f86d-f141-41...,Remote,Open,LinkedIn,5/2/2022,NaN,NaN,hn
124,Faire,NaN,Data Scientist New Graduate,https://boards.greenhouse.io/faire/jobs/607276...,Remote,Open,LinkedIn,5/2/2022,NaN,NaN,hn


In [77]:
nycdsa = df.groupby('Source')['Company'].nunique().reset_index().rename(columns={'Company' : 'Positions'})
fig = px.pie(nycdsa, values='Positions', names='Source', title='Jobs via Source')
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#151515', width=2)))
fig.show()

In [78]:
industry = df.groupby('Industry')['Company'].nunique().reset_index().rename(columns={'Company' : 'Jobs'}).sort_values(by='Jobs', ascending=False).head(10)

fig = px.bar(industry, title='Jobs by Industry (top 10)', x='Industry', y='Jobs')
fig.show()